In [ ]:
!pip install keras-swa==0.1.5

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue May 11 14:04:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    33W /  70W |   1088MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from swa.tfkeras import SWA
import tensorflow.keras.backend as K

In [ ]:
from keras.datasets import cifar10

In [ ]:
(trainX, trainy), (testX, testy) = cifar10.load_data()
trainX = trainX/255
testX = testX/255

In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K

In [ ]:
lb = LabelBinarizer()                                  #Binarize labels in a one-vs-all fashion.
trainy = lb.fit_transform(trainy)                     #Fit label binarizer and transform multi-class labels to binary labels
testy = lb.fit_transform(testy)

In [ ]:
def label_dict(img, labels):  # creating dictionary of labels
  lab_lis = {}
  for i in range(10):
    lab_lis[i] = []          #declaring key values as labels  in dictionary
  data = list(zip(img, labels))  # list of tuples of each image and label pair
  for im, lab in data:
    idx = np.argmax(lab)
    lab_lis[idx].append((im, lab))
  return lab_lis

In [ ]:
t = label_dict(trainX, trainy)

In [ ]:
unlab_data = []
unlab_img_data = []
unlab_img_lab = []
for i in range(len(t)):
  unlab_data.extend(t[i][4000:]) # extracting 1000 imgaes from each class in the training set to generate the pool of so called unlabeled set
for x,y in unlab_data:
  unlab_img_data.append(x)
  unlab_img_lab.append(y)

In [ ]:
client_names = ["client_"+str(i) for i in range(10)]  # makiing non_iid datas set for each client
client_data = {}
for client in client_names:
  client_data[client] = []
i = 0
for client in client_names:
  client_data[client].extend(t[i%10][:1960])
  client_data[client].extend(t[(i+1)%10][1960:3920])
  i += 1
for i in range(10):
  k = 0
  for j in range(10):
    if j != (i)%10 and j != (i+1)%10:
      client_data["client_"+str(j)].extend(t[i][3920 + 10*k: 3920 + 10*(k+1)])
      k += 1

In [ ]:
def batch(zip_data, batch_size = 40):
  data, label = zip(*zip_data)  # Unzipping the img and label pair 
  dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label))) # create a dataset object to make use of other transformation attributes
  return dataset.shuffle(len(label)).batch(batch_size)# shuffles data and creates batches of size 40

In [ ]:
client_batched = {}
for (client_name, data) in client_data.items():
  client_batched[client_name] = batch(data)

test_batched = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(len(testy))
unlab_dataset = tf.data.Dataset.from_tensor_slices((unlab_img_data)).batch(len(unlab_img_data))

In [ ]:
class MLP:
    @staticmethod
    def build():
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3))) # 32 filters of size 3*3
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(AveragePooling2D((2, 2))) # aveeraging the cell cvales for size of 2*2, witha stride of 2
        model.add(Dropout(0.2))  # randomly assign the value o to nodes, with probability of 0.2
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(AveragePooling2D((2, 2)))
        model.add(Dropout(0.2))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(AveragePooling2D((2, 2)))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dropout(0.2))
        model.add(Dense(10, activation='softmax'))
        return model

In [ ]:
model = MLP()
model = model.build()

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
average_pooling2d_3 (Average (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
average_pooling2d_4 (Average (None, 8, 8, 64)         

In [ ]:
t = model.get_weights()
for v in t:
  print(np.shape(v))

(3, 3, 3, 32)
(32,)
(3, 3, 32, 32)
(32,)
(3, 3, 32, 64)
(64,)
(3, 3, 64, 64)
(64,)
(3, 3, 64, 128)
(128,)
(3, 3, 128, 128)
(128,)
(2048, 128)
(128,)
(128, 10)
(10,)


In [ ]:
comms_round = 45
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=0.01, 
                momentum=0.9
               )   
optimizer2 = SGD(lr=0.001,
                 momentum=0.9
               )   

In [ ]:
# model = MLP()
# model = model.build()
# model.compile(loss=loss, 
#                       optimizer=optimizer2, 
#                       metrics=metrics)
# print((model.optimizer.lr))
# model2 = MLP()
# model2 = model2.build()
# model2.compile(loss=loss, 
#                       optimizer=optimizer, 
#                       metrics=metrics)
# print((model2.optimizer.lr))

In [ ]:
weight_scale = []
for (name, data) in client_data.items():
  weight_scale.append(len(data)/40000)
weight_scale = np.array(weight_scale).reshape((len(weight_scale),1))

In [ ]:
# def scale_model_weights(weight, scalar):
#     weight_final = []
#     steps = len(weight)
#     for i in range(steps):
#         weight_final.append(scalar * weight[i])
#     return weight_final

def sum_scaled_weights(scaled_weight_list, M, unlab_data):
    #print(type(scaled_weight_list[0][0]))
    avg_grad_mean = list()
    avg_grad_var = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        #print(np.shape(grad_list_tuple))
        if len(np.shape(grad_list_tuple)) == 5:
          w = weight_scale.reshape((len(weight_scale), 1, 1, 1, 1))
        elif len(np.shape(grad_list_tuple)) == 4:
          w = weight_scale.reshape((len(weight_scale), 1, 1, 1))
        elif len(np.shape(grad_list_tuple)) == 3:
          w = weight_scale.reshape((len(weight_scale), 1, 1))
        else:
          w = weight_scale
        layer_mean = np.sum(grad_list_tuple*w, axis=0)
        #print(np.shape(layer_mean))
        layer_variance = np.sum(np.power(grad_list_tuple - layer_mean, 2)*w, axis = 0) 
        avg_grad_mean.append(layer_mean)
        avg_grad_var.append(layer_variance)
    return avg_grad_mean


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
mlp_global = MLP()
global_model = mlp_global.build()

for comm_round in range(comms_round):
            
    
    global_weights = global_model.get_weights()
    
    
    scaled_local_weight_list = list()
    client_names = list(client_batched.keys())
    random.shuffle(client_names)
    
    for client in client_names:
        def scheduler(epoch, lr):
            if epoch  == 8:
              lr = lr*0.1
            if epoch == 15:
              lr = lr*0.1  
            return lr
        mlp_local = MLP()
        local_model = mlp_local.build()
        local_model.compile(loss=loss, 
                      optimizer=SGD(lr=0.01,momentum=0.9), 
                      metrics=metrics)
        local_model.set_weights(global_weights)
        
        callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        local_model.fit(client_batched[client], epochs=25, verbose=0, callbacks=[callback])
        scaled_local_weight_list.append(local_model.get_weights())
        
        K.clear_session()
    avg = sum_scaled_weights(scaled_local_weight_list, 10, unlab_dataset)
    mlp_global = MLP()
    global_model = mlp_global.build()
    global_model.compile(loss=loss, 
                      optimizer=SGD(lr=0.001,momentum=0.9), 
                      metrics=metrics)
    global_model.set_weights(avg)

    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

comm_round: 0 | global_acc: 13.630% | global_loss: 2.300297975540161
comm_round: 1 | global_acc: 26.850% | global_loss: 2.274691581726074
comm_round: 2 | global_acc: 42.240% | global_loss: 2.1918423175811768
comm_round: 3 | global_acc: 48.610% | global_loss: 2.109875440597534
comm_round: 4 | global_acc: 53.950% | global_loss: 2.0326969623565674
comm_round: 5 | global_acc: 57.250% | global_loss: 1.9741109609603882
comm_round: 6 | global_acc: 59.710% | global_loss: 1.9329475164413452
comm_round: 7 | global_acc: 61.590% | global_loss: 1.902298092842102
comm_round: 8 | global_acc: 62.970% | global_loss: 1.8830710649490356
comm_round: 9 | global_acc: 63.640% | global_loss: 1.8627345561981201
comm_round: 10 | global_acc: 64.710% | global_loss: 1.8530644178390503
comm_round: 11 | global_acc: 65.050% | global_loss: 1.8445842266082764
comm_round: 12 | global_acc: 65.790% | global_loss: 1.8380894660949707
comm_round: 13 | global_acc: 66.110% | global_loss: 1.8324648141860962
comm_round: 14 | glo